1. real dataset
1. link the path
    1. analytical 
    1. statistical
    - measure the accuracy
    - marriage matching algorithm 
what you want to maximize

### Analytical

In [1]:
import sqlite3
import pandas as pd
import math
import networkx as nx
import json
import numpy as np
from multiprocessing import Pool
def unpack(row):
    return row['walker_id'], row['time'], float(row['x']), float(row['y'])

In [2]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_3000.db')

df_walks = pd.read_sql_query("SELECT * FROM walks", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)

In [3]:
ids = df_v['walker_id'].tolist()

In [4]:
df_v.head()

,walker_id,time,x_json,y_json,ratio
0,774f6377b5,0,"{""mean"":11.878909087200725,""dev"":3,""v"":15.9110...","{""mean"":30.86683282944279,""dev"":3,""v"":31.75124...",0.97
1,fa3cd5a96f,2,"{""mean"":14.589745907757557,""dev"":3,""v"":14.2403...","{""mean"":29.005541277848653,""dev"":3,""v"":30.8072...",0.97
2,b8fb2f2cdb,5,"{""mean"":16.799026624679943,""dev"":3,""v"":18.7525...","{""mean"":29.480759844067144,""dev"":3,""v"":33.9879...",0.97
3,8e2ad4c380,6,"{""mean"":18.75254138338596,""dev"":3,""v"":16.60640...","{""mean"":33.98795510979555,""dev"":3,""v"":31.65725...",0.97
4,8e2ad4c380,7,"{""mean"":16.606406428772626,""dev"":3,""v"":13.3611...","{""mean"":31.657252568144827,""dev"":3,""v"":28.1651...",0.97


In [11]:
def get_probability(x, mean, sd):
    x = x
    var = float(sd)**2
    denom = (2*math.pi*var)**.5
    num = math.exp(-(float(x)-float(mean))**2/(2*var))
    return num/denom

In [6]:
def find(row):
    walker_id, time, x, y = row

    v_row = df_v.loc[(df_v['walker_id'] == walker_id) & (df_v['time'] == time)].iloc[0]

    x_json, y_json, ratio = json.loads(v_row['x_json']), json.loads(v_row['y_json']), v_row['ratio']

    vx = (x - x_prev) / 10 / ratio
    vy = (y - y_prev) / 10 / ratio
    p = get_probability(vx, x_json['mean'], x_json['dev']) * get_probability(vy, y_json['mean'], y_json['dev'])

    return (p, walker_id)

In [22]:
matching = {}
i = -1

while True:
    i += 1
    prev = df_walks.loc[df_walks['time'] == str(i)]
    curr = df_walks.loc[df_walks['time'] == str(i+1)]
    if prev.shape[0] == 0:
        break
    for _, row_prev in prev.iterrows():
        walker_id_prev, time_prev, x_prev, y_prev = unpack(row_prev)
        
        rows = []
        for _, row in curr.iterrows():
            walker_id, time, x, y = unpack(row)
            rows.append([walker_id, time, x, y])

        pool = Pool(processes=10)
        max_p, next_id = 0, None
        results = pool.imap_unordered(find, rows)
        pool.close()
        
        for p, walker_id in results:
#             print(p, walker_id)
            if p > max_p:
                max_p, next_id = p, walker_id
#         for _, row in curr.iterrows():
#             walker_id, time, x, y = unpack(row)

#             v_row = df_v.loc[(df_v['walker_id'] == walker_id) & (df_v['time'] == time)].iloc[0]
            
#             x_json, y_json, ratio = json.loads(v_row['x_json']), json.loads(v_row['y_json']), v_row['ratio']

#             vx = (x - x_prev) / 10 / ratio
#             vy = (y - y_prev) / 10 / ratio
#             p = get_probability(vx, x_json['mean'], x_json['dev']) * get_probability(vy, y_json['mean'], y_json['dev'])
            
#             if p > max_p:
#                 max_p, next_id = p, walker_id

        matching[walker_id_prev] = next_id


In [23]:
graph = nx.Graph([(i, j) for i, j in matching.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)

In [25]:
df_walks_connected = pd.DataFrame(df_walks)
for i, component in enumerate(connected_components):
    new_id = i
    for c in component:
        df_walks_connected = df_walks_connected.replace(c, new_id)

In [26]:
cnx.execute('DROP TABLE IF EXISTS walks_attached')
df_walks_connected.to_sql('walks_attached', con=cnx)

### Statistical

In [2]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_30000.db')

df_walks = pd.read_sql_query("SELECT * FROM walks JOIN walkers ON walkers.id = walks.walker_id", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_v = pd.read_sql_query("SELECT * FROM velocitys", cnx)

In [3]:
df_walks.head()

,walker_id,time,x,y,json,id,real_id,resolved
0,6c941deb58,0,361.274388646454,93.7774943499492,"{""resolved"":false,""location"":{""latitude"":60.16...",6c941deb58,0,0
1,6c941deb58,1,671.463623372485,179.370863352698,"{""resolved"":false,""location"":{""latitude"":60.16...",6c941deb58,0,0
2,6c4f36901d,5,1805.66148473744,550.615488790935,"{""resolved"":false,""location"":{""latitude"":60.16...",6c4f36901d,0,0
3,4fcb6a8f8d,3,1249.32907878941,355.013187814622,"{""resolved"":false,""location"":{""latitude"":60.16...",4fcb6a8f8d,0,0
4,e56fa980de,7,2447.0202419566,738.863496964905,"{""resolved"":false,""location"":{""latitude"":60.16...",e56fa980de,0,0


In [4]:
ids = df_v['walker_id'].tolist()
real_ids = set(df_walks['real_id'].tolist())

In [5]:
n_total_x, n_total_y = 0, 0
v_dic_x = {}
v_dic_y = {}
for real_id in real_ids:
    walk = df_walks.loc[df_walks['real_id'] == real_id]

    for time in range(walk.shape[0]-1):
        prev = walk.loc[walk['time'] == str(time)]
        curr = walk.loc[walk['time'] == str(time + 1)]
        
        vx = round(float(curr['x']) - float(prev['x']), 0)
        vy = round(float(curr['y']) - float(prev['y']), 0)

        if vx in v_dic_x:
            v_dic_x[vx] += 1
        else:
            v_dic_x[vx] = 1
        n_total_x += 1
        
        if vy in v_dic_y:
            v_dic_y[vy] += 1
        else:
            v_dic_y[vy] = 1
        n_total_y += 1

In [6]:
for k, v in v_dic_x.items():
    v_dic_x[k] = (v + 1) / (n_total_x + len(v_dic_x.keys()))
for k, v in v_dic_y.items():
    v_dic_y[k] = (v + 1) / (n_total_y + len(v_dic_y.keys()))

In [7]:
print((n_total_x + len(v_dic_x.keys())), (n_total_y + len(v_dic_y.keys())))

30770 30818


In [8]:
cnx = sqlite3.connect('../corona-sniffer/backend/data/database_3000.db')

df_walks = pd.read_sql_query("SELECT * FROM walks", cnx)
df_walkers = pd.read_sql_query("SELECT * FROM walkers", cnx)
df_v = pd.read_sql_query("SELECT * FROM velocitys", cnx)

In [9]:
def find(row):
    walker_id, time, x, y = row

    v_row = df_v.loc[(df_v['walker_id'] == walker_id) & (df_v['time'] == time)].iloc[0]

    x_json, y_json, ratio = json.loads(v_row['x_json']), json.loads(v_row['y_json']), v_row['ratio']

    vx = round(x - x_prev, 0)
    vy = round(y - y_prev, 0)
    if vx in v_dic_x:
        px = v_dic_x[v]
    else:
        px = 1 / (n_total_x + len(v_dic_x.keys()))
    if vy in v_dic_y:
        py = v_dic_y[v]
    else:
        py = 1 / (n_total_y + len(v_dic_y.keys()))
    return (px * py, walker_id, (x - x_prev, y - y_prev))

#         dis = float('inf')
#         key = None
#         for k, val in v_dic.items():
#             new_dis = ((k[0] - v[0]) ** 2 + (k[1] - v[1]) ** 2)
#             if dis > new_dis:
#                 dis = new_dis
#                 key = k
#         p = v_dic[key]

In [10]:
matching = {}
for i in range(100):
    prev = df_walks.loc[df_walks['time'] == str(i)]
    curr = df_walks.loc[df_walks['time'] == str(i+1)]
    
    for _, row_prev in prev.iterrows():
        walker_id_prev, time_prev, x_prev, y_prev = unpack(row_prev)
        visited = set()
        
        rows = []
        for _, row in curr.iterrows():
            walker_id, time, x, y = unpack(row)
            rows.append([walker_id, time, x, y])

        pool = Pool(processes=10)
        max_p, next_id = 0, []
        results = pool.imap_unordered(find, rows)
        pool.close()
        
        
        for p, walker_id, (vx, vy) in results:
            if p > max_p:
                max_p, next_id = p, [(walker_id, vx, vy)]
            elif p == max_p:
                max_p = p
                next_id.append((walker_id, vx, vy))
        
        list.sort(next_id, key=lambda x: np.linalg.norm([x[1]-x_prev, x[2]-y_prev]))
        for (id, _, _) in next_id:
            if id not in visited:
                visited.add(id)
                matching[walker_id_prev] = id
                break


In [11]:
graph = nx.Graph([(i, j) for i, j in matching.items() if i and j])
connected_components = sorted(nx.connected_components(graph), key=len, reverse=True)

In [12]:
real_ids = df_walkers["real_id"].tolist()
id_match = {real_id: set() for real_id in real_ids}
for real_id in real_ids:
    [id_match[real_id].add(id) for id in df_walkers.loc[df_walkers["real_id"] == real_id]['id'].tolist()]

In [14]:
df_walks_connected = pd.DataFrame(df_walks)
df_walks_connected['old_id'] = [None] * df_walks_connected.shape[0]

In [15]:
for i, component in enumerate(connected_components):
    new_id = i
    time_set = set()
    for c in component:
        rows = df_walks_connected.loc[df_walks_connected['walker_id'] == c]
        
        cyclic = False
        for time in rows['time'].tolist():
            if time in time_set:
                cyclic = True
                break
            else:
                time_set.add(time)
        if cyclic:
            continue
        else:
            tmp = c
            df_walks_connected = df_walks_connected.replace(c, new_id)
            rows['old_id'] = c

In [65]:
cnx.execute('DROP TABLE IF EXISTS walks_attached')
df_walks_connected.to_sql('walks_attached', con=cnx)

In [66]:
df_walks_connected.loc[df_walks_connected['walker_id'] == 2]

,walker_id,time,x,y,json
396,2,2,-2678.11577056954,-1780.7346603135,"{""resolved"":false,""location"":{""latitude"":60.14..."
397,2,3,-2616.47114827375,-1898.91182449137,"{""resolved"":false,""location"":{""latitude"":60.14..."
398,2,4,-2504.21295019204,-1972.28830260992,"{""resolved"":false,""location"":{""latitude"":60.14..."
399,2,5,-2384.52318790835,-2020.353699141,"{""resolved"":false,""location"":{""latitude"":60.14..."
504,2,10,-2584.56766911641,-2440.35364111122,"{""resolved"":false,""location"":{""latitude"":60.14..."
...,...,...,...,...,...
1365,2,76,-11388.1469979293,-11026.3472681642,"{""resolved"":false,""location"":{""latitude"":60.06..."
1700,2,6,-1764.96432377177,-1610.75748665389,"{""resolved"":false,""location"":{""latitude"":60.14..."
1701,2,7,-1900.37161552083,-1653.28283855725,"{""resolved"":false,""location"":{""latitude"":60.14..."
2094,2,0,-1708.79250721564,-1734.06592542844,"{""resolved"":false,""location"":{""latitude"":60.14..."


In [68]:
len(set(df_walks_connected['walker_id'].tolist()))

182